In [69]:
import os
import shutil
import json
import xmltodict
import pprint
from tqdm import tqdm

from collections import defaultdict

In [2]:
folder_path = "/home/harish3110/document-ai/data/nanonets_new_annotations"

xml_folder_path = os.path.join(folder_path, "Annotations")
img_folder_path = os.path.join(folder_path, "ImageSets")

xml_files = os.listdir(xml_folder_path)
img_files = os.listdir(img_folder_path)

In [54]:
def copy_file(source_path, destination_path):
    try:
        shutil.copy(source_path, destination_path)
    except IOError as e:
        print(f"Unable to copy file. {e}")

def convert_xml_to_dict(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data_dict = xmltodict.parse(file.read())
        return data_dict
    except IOError as e:
        print(f"Unable to convert file. {e}")
        return None
        

In [56]:
# Create folders for intersection of xml and img files
img_path = os.path.join(folder_path, "imgs")
xml_path = os.path.join(folder_path, "xmls")

os.makedirs(img_path, exist_ok=True)
os.makedirs(key_path, exist_ok=True)

for xml_file in tqdm(xml_files):
    if xml_file.endswith('.xml'):
        img_file = xml_file[:-4] + ".jpeg"
        if img_file in img_files:
            copy_file(os.path.join(img_folder_path, img_file), os.path.join(img_path, img_file))
            copy_file(os.path.join(xml_folder_path, xml_file), os.path.join(xml_path, xml_file))

100%|██████████| 6508/6508 [00:03<00:00, 2108.14it/s]


In [155]:
# Convert xml to json and store in keys folder

ref_count = defaultdict(lambda: {"count": 0, "files": []})
key_path = os.path.join(folder_path, "keys")
os.makedirs(key_path, exist_ok=True)

xml_new_files = os.listdir(xml_path)

for xml in tqdm(xml_new_files):
    result = {}
    data = convert_xml_to_dict(os.path.join(xml_path, xml))['annotation']['object']
    json_file_name = os.path.splitext(xml)[0] + '.json'
    if type(data) == dict:
        name = data['name']
        ocr_text = data['ocr_text']
        result[name] = ocr_text
    
    elif type(data) == list:    
        for obj in data:
            name = obj['name']
            ocr_text = obj['ocr_text']

            if name != None:
                if name not in result:
                    result[name] = ocr_text
                else:
                    if isinstance(result[name], list):
                        result[name].append(ocr_text)
                    else:
                        result[name] = [result[name], ocr_text]
                    ref_count[name]['count'] += 1
                    ref_count[name]['files'].append(json_file_name)

    # Save dictionary as JSON
    json_data = json.dumps(result, indent=4)
    json_file_path = os.path.join(key_path, json_file_name)
    with open(json_file_path, 'w') as json_file:
        json_file.write(json_data)

100%|██████████| 6493/6493 [00:02<00:00, 2505.52it/s]


In [156]:
ref_count

defaultdict(<function __main__.<lambda>()>,
            {'payment_schedule_date': {'count': 7093,
              'files': ['cd2dd9f0-ab19-46c6-82d5-47909f127b8f-1.json',
               'cd2dd9f0-ab19-46c6-82d5-47909f127b8f-1.json',
               'cd2dd9f0-ab19-46c6-82d5-47909f127b8f-1.json',
               'cd2dd9f0-ab19-46c6-82d5-47909f127b8f-1.json',
               'cd2dd9f0-ab19-46c6-82d5-47909f127b8f-1.json',
               'cd2dd9f0-ab19-46c6-82d5-47909f127b8f-1.json',
               'cd2dd9f0-ab19-46c6-82d5-47909f127b8f-1.json',
               'cd2dd9f0-ab19-46c6-82d5-47909f127b8f-1.json',
               'cd2dd9f0-ab19-46c6-82d5-47909f127b8f-1.json',
               'cd2dd9f0-ab19-46c6-82d5-47909f127b8f-1.json',
               'cd2dd9f0-ab19-46c6-82d5-47909f127b8f-1.json',
               'b9e4848b-1512-4c6f-8528-3ad77f551c12-1.json',
               'b9e4848b-1512-4c6f-8528-3ad77f551c12-1.json',
               'b9e4848b-1512-4c6f-8528-3ad77f551c12-1.json',
               'b9e4848b-

In [157]:
count = 0
for key, value in ref_count.items():
    if key != 'payment_schedule_date':
        if value['count'] > 0:
            print(key, value['count'])
            count += value['count']

print(count)

policy_type 100
premium 48
document_type 13
customer_name 117
customer_address 11
document_type_description 1
broker_address 12
broker_name 7
copy_type 2
policy_number 8
expiry_date 3
effective_document_date 6
insurance_carrier_address 2
insurance_carrier 6
effective_date 1
amount_due 2
339


In [110]:
ref_count['premium']['files'][1]

'239d7330-fc65-41b1-9322-8a1945c8f00b-2.json'

In [115]:
key = 'policy_type'

for file in ref_count[key]['files']:
    json_file_path = os.path.join(key_path, file)
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)
    print(data[key])

['PERSONAL PROPERTY', '39659590']
['Property', 'Homeowners']
['Property', 'Homeowners']
['Property', 'Homeowners']
['Off Road Vehicle Policy', 'Property']
['Property', 'Homeowners']
['Property', 'Homeowners']
['Property', 'Enhanced Homeowners']
['Property', 'Homeowners']
['Property', 'Enhanced Homeowners']
['Property', 'Homeowners']
['Property', 'Homeowners']
['Property', 'Condominium Unit Owners']
['Off Road Vehicle Policy', 'Property']
['Property', 'Homeowners']
['Property', 'Homeowners']
['Property', 'Homeowners']
['Property', 'Enhanced Homeowners']
['Property', 'Enhanced Homeowners']
['Property', 'Enhanced Homeowners']
['Property', 'Enhanced Homeowners']
['Property', 'Homeowners']
['Property', 'Homeowners']
['Property', 'Homeowners']
['Property', 'Homeowners']
['Property', 'Homeowners']
['Property', 'Homeowners']
['Property', 'Enhanced Homeowners']
['Property', 'Enhanced Homeowners']
['Property', 'Condominium Unit Owners']
['Property', 'Homeowners']
['FARM PACKAGE \nPOLICY', '26796

In [126]:
folder_path = '/home/harish3110/nanonets-annotations/nanonets-annotations/to-annotate'

for key in tqdm(ref_count.keys()):
    if key != 'payment_schedule_date':
        output_folder = os.path.join(folder_path, f"multiple_{key}")
        os.makedirs(output_folder, exist_ok=True)
        for file in ref_count[key]['files']:
            file_name = os.path.splitext(file)[0] + '.xml'
            shutil.copy(os.path.join(xml_folder_path, file_name), os.path.join(output_folder, file_name))


100%|██████████| 17/17 [00:00<00:00, 367.61it/s]


In [158]:
def convert_to_json(xml_file_path, output_folder):
    result = {}
    data = convert_xml_to_dict(xml_file_path)['annotation']['object']
    if type(data) == dict:
        name = data['name']
        ocr_text = data['ocr_text']
        result[name] = ocr_text
    
    elif type(data) == list:    
        for obj in data:
            name = obj['name']
            ocr_text = obj['ocr_text']

            if name != None:
                if name not in result:
                    result[name] = ocr_text
                else:
                    if isinstance(result[name], list):
                        result[name].append(ocr_text)
                    else:
                        result[name] = [result[name], ocr_text]

    # Save dictionary as JSON
    json_data = json.dumps(result, indent=4)
    file_name = os.path.splitext(os.path.basename(xml_file_path))[0] + '.json'
    json_file_path = os.path.join(output_folder, file_name)
    with open(json_file_path, 'w') as json_file:
        json_file.write(json_data)


In [160]:
for folder in os.listdir(folder_path):
    output_folder = os.path.join(folder_path, folder, 'json-files')
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
    os.makedirs(output_folder, exist_ok=True)
    xml_files = os.listdir(os.path.join(folder_path, folder))
    for file in xml_files:
        if file != 'json-files':
            file_to_convert = os.path.join(folder_path, folder, file)
            convert_to_json(file_to_convert, output_folder)
            


ExpatError: not well-formed (invalid token): line 1, column 0